<a href="https://colab.research.google.com/github/mzrd91/CMA-ES-Algorithm-for-PEMFC-Optimization/blob/main/CMA_ES_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tabulate import tabulate

class CMAES:
    def __init__(self, num_params, population_size, sigma_init=0.5, param_min=None, param_max=None):
        self.num_params = num_params
        self.population_size = population_size
        self.sigma = sigma_init
        self.mean = np.random.rand(num_params)
        self.C = np.eye(num_params)
        self.weights = np.log(population_size / 2 + 1) - np.log(np.arange(1, population_size + 1))
        self.weights /= np.sum(self.weights)
        self.mu = population_size // 2
        self.weights = self.weights[:self.mu]
        self.mueff = np.sum(self.weights) ** 2 / np.sum(self.weights ** 2)
        self.cs = (self.mueff + 2) / (num_params + self.mueff + 5)
        self.damps = 1 + 2 * max(0, np.sqrt((self.mueff - 1) / (num_params + 1)) - 1) + self.cs
        self.cc = 4 / (num_params + 4)
        self.c1 = 2 / ((num_params + 1.3) ** 2 + self.mueff)
        self.cmu = min(1 - self.c1, 2 * (self.mueff - 2 + 1 / self.mueff) / ((num_params + 2) ** 2 + self.mueff))
        self.pc = np.zeros(num_params)
        self.ps = np.zeros(num_params)
        self.B = np.eye(num_params)
        self.D = np.ones(num_params)
        self.inv_sqrt_C = self.B @ np.diag(1 / self.D) @ self.B.T
        self.eigeneval = 0
        self.counteval = 0
        self.chiN = np.sqrt(num_params) * (1 - 1 / (4 * num_params) + 1 / (21 * num_params ** 2))

        # Parameter bounds
        self.param_min = param_min if param_min is not None else np.full(num_params, -np.inf)
        self.param_max = param_max if param_max is not None else np.full(num_params, np.inf)

    def sample_population(self):
        Z = np.random.normal(size=(self.population_size, self.num_params))
        sampled_population = self.mean + self.sigma * Z @ self.B @ np.diag(self.D)
        # Apply constraints to the sampled population
        return np.clip(sampled_population, self.param_min, self.param_max)


    def update(self, X, fitness_values):
        self.counteval += len(X)
        arindex = np.argsort(fitness_values)
        arx = X[arindex[:self.mu]]
        arz = (arx - self.mean) / self.sigma
        self.mean = np.dot(self.weights, arx)
        zmean = np.dot(self.weights, arz)
        self.ps = (1 - self.cs) * self.ps + np.sqrt(self.cs * (2 - self.cs) * self.mueff) * self.inv_sqrt_C @ zmean
        hsig = np.linalg.norm(self.ps) / np.sqrt(1 - (1 - self.cs) ** (2 * self.counteval / self.population_size)) / self.chiN < 1.4 + 2 / (self.num_params + 1)
        self.pc = (1 - self.cc) * self.pc + hsig * np.sqrt(self.cc * (2 - self.cc) * self.mueff) * zmean
        self.C = ((1 - self.c1 - self.cmu) * self.C
                 + self.c1 * (self.pc @ self.pc.T + (1 - hsig) * self.cc * (2 - self.cc) * self.C)
                 + self.cmu * arz.T @ np.diag(self.weights) @ arz)
        self.sigma *= np.exp((self.cs / self.damps) * (np.linalg.norm(self.ps) / self.chiN - 1))
        if self.counteval - self.eigeneval > self.population_size / (self.c1 + self.cmu) / self.num_params / 10:
            self.eigeneval = self.counteval
            self.C = np.triu(self.C) + np.triu(self.C, 1).T
            self.D, self.B = np.linalg.eigh(self.C)
            self.D = np.sqrt(self.D)
            self.inv_sqrt_C = self.B @ np.diag(1 / self.D) @ self.B.T
            self.chiN = np.sqrt(self.num_params) * (1 - 1 / (4 * self.num_params) + 1 / (21 * self.num_params ** 2))


    def optimize(self, fitness_function, max_generations):
        best_fitness_per_generation = []
        for generation in range(max_generations):
            X = self.sample_population()
            fitness_values = np.array([fitness_function(x) for x in X])
            self.update(X, fitness_values)


            best_fitness_per_generation.append(min(fitness_values))

        return self.mean, best_fitness_per_generation

def pemfc_fitness(params):
    # Unpacking the parameters
    x1, x2, x3, x4, l, RC, B = params

    # Constants and parameters from the Energy article
    T = 353.15  # Temperature in Kelvin
    pO2 = 5  # Partial pressure of oxygen in bar
    CO2 = 5.08 * pO2 * 1e6 * np.exp(-498 / T)  # Concentration of CO2
    i = 0.1  # Current (A), this can be varied or set as needed
    A = 27  # Cell active area in cm^2
    ilimit_den = 860  # Limiting current density mA/cm^2
    iden = i / A  # Current density A/cm^2

    # Calculating losses
    hact = -x1 + x2 * T + x3 * T * np.log(CO2) + x4 * T * np.log(i)  # Activation loss
    hconc = -B * np.log(1 - iden / ilimit_den)  # Concentration loss
    rM = 181 * l - 0.1634 + 0.062 * (4 / ((T / 303) - 1)) ** 2.5  # Specific resistivity
    RM = rM * l / A  # Equivalent resistance of membrane
    hohm = i * (RM + RC)

    Nernst_potential = 1.229 # Nernst potential is assumed to be 1.229 V
    Vcell = Nernst_potential - hact - hohm - hconc

    total_loss = hact + hohm + hconc
    fitness = 1 / total_loss

    return fitness

param_min = np.array([-1.19969, 0.001, 3.0e-5, -1.98e-4, 14, 0.0001, 0.016])
param_max = np.array([-0.8532, 0.005, 9.8e-5, -9.54e-5, 24, 0.0008, 0.5])

all_optimized_params = np.zeros((30, 7))  # 30 runs, 7 parameters each
first_and_last_fitness = []
all_best_fitnesses = []

for run in range(30):
    cma_es = CMAES(num_params=7, population_size=50, param_min=param_min, param_max=param_max)
    optimized_params, best_fitnesses = cma_es.optimize(pemfc_fitness, max_generations=100)
    all_optimized_params[run] = optimized_params
    all_best_fitnesses.append(best_fitnesses[-1])
    first_and_last_fitness.append((best_fitnesses[0], best_fitnesses[-1]))

# 1st and last Generation's fitness, to check if fitness are working (Change the run < 30, can show all fitnesses)
for run, (first_fitness, last_fitness) in enumerate(first_and_last_fitness):
    if run < 1:
        print(f"Generation 1: Best fitness = {first_fitness}")
        print(f"Generation 30: Best fitness = {last_fitness}")

mean_params = np.mean(all_optimized_params, axis=0)
std_params = np.std(all_optimized_params, axis=0)
variance_params = np.var(all_optimized_params, axis=0)


# Display the summary as a table
table_header = ["Parameter", "Upper Bound", "Lower Bound", "Mean", "Std", "Variance"]
table_rows = []
for i, (param_name, lower, upper) in enumerate(zip(["x1", "x2", "x3", "x4", "l", "RC", "B"], param_min, param_max)):
    table_rows.append([param_name, upper, lower, mean_params[i], std_params[i], variance_params[i]])

print(tabulate(table_rows, headers=table_header, tablefmt="grid"))

# Print the average best fitness across runs(I print this as Optional)
average_best_fitness = np.mean(all_best_fitnesses)
print(f"\nAverage Best Fitness across 30 runs: {average_best_fitness}")


Generation 1: Best fitness = 0.006947524898537756
Generation 30: Best fitness = 0.0024654182189465133
+-------------+---------------+---------------+--------------+-------------+-------------+
| Parameter   |   Upper Bound |   Lower Bound |         Mean |         Std |    Variance |
+=============+===============+===============+==============+=============+=============+
| x1          |     -0.8532   |     -1.19969  | -1.94319     | 0.00899596  | 8.09273e-05 |
+-------------+---------------+---------------+--------------+-------------+-------------+
| x2          |      0.005    |      0.001    |  0.00809168  | 7.67762e-05 | 5.89458e-09 |
+-------------+---------------+---------------+--------------+-------------+-------------+
| x3          |      9.8e-05  |      3e-05    |  0.000142913 | 6.26424e-06 | 3.92407e-11 |
+-------------+---------------+---------------+--------------+-------------+-------------+
| x4          |     -9.54e-05 |     -0.000198 | -0.000267372 | 2.08226e-05 | 4.